In [3]:
import os
import sys
import re
import matplotlib
import pandas as pd
import numpy as np
from os.path import splitext
import ipaddress as ip
import tldextract
import whois
import datetime
from urllib.parse import urlparse
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
import csv

C:\Users\Cxovrika\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [9]:
cnt = 0
with open('malicious.txt','r', encoding='utf-8') as legitimate:
    for rec in legitimate:
        cnt = cnt+1
        if rec == np.nan:
            print(1)
        #with open('legitimate_words.csv','a', encoding='utf-8') as legitimate_words:
            #words = re.sub("[^A-Za-z0-9]", " ", rec)
            #words = re.sub(' +',' ',words)
            #words = words.strip()
            #words = words.lower()
            #legitimate_words.write( words + '\n')

In [10]:
cnt

48126

In [3]:
with open('malicious.txt','r', encoding='utf-8') as malicious:
    for rec in malicious:
        with open('malicious_words.csv','a', encoding='utf-8') as malicious_words:
            words = re.sub("[^A-Za-z0-9]", " ", rec)
            words = re.sub(' +',' ',words)
            words = words.strip()
            words = words.lower()
            malicious_words.write( words + '\n')

In [2]:
"hello world sample text".split(" ")

['hello', 'world', 'sample', 'text']

In [3]:
all_words = []

with open('malicious_words.csv','r', encoding='utf-8') as malicious_words:
    for row in malicious_words:
        all_words.extend(row.rstrip().split(" "))
        

with open('legitimate_words.csv','r', encoding='utf-8') as legitimate_words:
    for row in legitimate_words:
        all_words.extend(row.rstrip().split(" "))

In [3]:
rows = []
with open('legitimate_words.csv','r', encoding='utf-8') as malicious_words:
    for row in malicious_words:
        rows.append(row)

#print(rows)

In [5]:
from collections import Counter
all_word_counts = Counter(all_words)

#all_word_counts = all_word_counts.most_common()

dictionary = all_word_counts.most_common(20000)
for idx,row in enumerate(dictionary):
    with open('dictionary.csv','a', encoding='utf-8') as dictionary_file:
        dictionary_file.write(str(idx) + "," + row[0] + "," + str(row[1]) + "\n")
    

In [6]:
def str_to_input_data(input_text):
    
    row_count = sum(1 for line in open('dictionary.csv'))
    ans = np.zeros((row_count,1))
    for word in input_text.split(" "):
        with open('dictionary.csv', 'r') as csvfile:
            for row in csv.reader(csvfile):
                if row[1] == word:
                    ans[int(row[0])] = ans[int(row[0])]+1
            
    return ans

In [16]:
x = str_to_input_data("php php 20 javascript javascript striker olivia olivia")
print(type(x.shape))

<class 'tuple'>


In [15]:
legitimate_row_count = sum(1 for line in open('legitimate_words.csv'))
#X_Legitimate = np.zeros((20000,legitimate_row_count))

with open('legitimate_words.csv', 'r') as legitimate_words:
    for words in legitimate_words:
        str_to_input_data(input_text)
        X_Legitimate[:][row[0]-1] = str_to_input_data(row[1]) 

MemoryError: 

In [37]:
import pandas
import random

#n = 1000000 #number of records in file
#s = 50000 #desired sample size
#skip = sorted(random.sample(range(n),n-s))
#df_legitimate = pandas.read_csv('legitimate_words.csv', skiprows=skip,header=None)

df_legitimate = pd.read_csv('legitimate_words.csv',header=None)
df_malicious  = pd.read_csv('malicious_words.csv',header=None)
df_legitimate.insert(1,'new',1)
df_malicious.insert(1,'new',0)

In [38]:
df_legitimate.head()

,0,new
0,103886,1
1,rcanimal,1
2,458010b88d9ce,1
3,cclogovs,1
4,using localization,1


In [39]:
frames = [df_legitimate, df_malicious]
df = pd.concat(frames)
df = df[~df[0].isnull()]

In [40]:
df_legitimate.count()

0      1294530
new    1294530
dtype: int64

In [41]:
df_malicious.count()

0      47855
new    47866
dtype: int64

In [42]:
df.count()

0      1342385
new    1342385
dtype: int64

In [43]:
df_x = df[0]
df_y = df['new']

In [44]:
df_y.head()

0    1
1    1
2    1
3    1
4    1
Name: new, dtype: int64

In [45]:
from sklearn.feature_extraction.text import CountVectorizer
cv1 = CountVectorizer()

In [46]:
x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.2, random_state=4)

In [47]:
print(y_train)

1291128    1
245647     1
781429     1
708945     1
350794     1
160471     1
426013     1
405201     1
1165570    1
77054      1
43392      0
840001     1
621586     1
1242777    1
1008863    1
922184     1
482147     1
354212     1
118321     1
423884     1
1043714    1
748782     1
1103891    1
334568     1
1270385    1
622731     1
643996     1
1199244    1
820778     1
506965     1
          ..
743767     1
986042     1
1060162    1
478856     1
115742     1
472437     1
922306     1
947709     1
846246     1
712853     1
707093     1
1151159    1
549888     1
9603       1
722484     1
860138     1
390270     1
1186860    1
369586     1
646820     1
950120     1
226231     1
764727     1
860007     1
1156154    1
639432     1
653672     1
120705     1
991406     1
1000570    1
Name: new, Length: 1073908, dtype: int64


In [48]:
x_train.head()

1291128          holdingpage
245647                041205
781429           fish killer
708945     javascript mac ws
350794                225610
Name: 0, dtype: object

In [49]:
x_traincv=cv1.fit_transform(x_train)

In [50]:
x_traincv

<1073908x642165 sparse matrix of type '<class 'numpy.int64'>'
	with 2384999 stored elements in Compressed Sparse Row format>

In [51]:
x_testcv=cv1.transform(x_test)

In [52]:
mnb = MultinomialNB()

In [53]:
y_train=y_train.astype('int')

In [54]:
y_train

1291128    1
245647     1
781429     1
708945     1
350794     1
160471     1
426013     1
405201     1
1165570    1
77054      1
43392      0
840001     1
621586     1
1242777    1
1008863    1
922184     1
482147     1
354212     1
118321     1
423884     1
1043714    1
748782     1
1103891    1
334568     1
1270385    1
622731     1
643996     1
1199244    1
820778     1
506965     1
          ..
743767     1
986042     1
1060162    1
478856     1
115742     1
472437     1
922306     1
947709     1
846246     1
712853     1
707093     1
1151159    1
549888     1
9603       1
722484     1
860138     1
390270     1
1186860    1
369586     1
646820     1
950120     1
226231     1
764727     1
860007     1
1156154    1
639432     1
653672     1
120705     1
991406     1
1000570    1
Name: new, Length: 1073908, dtype: int32

In [55]:
mnb.fit(x_traincv,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [56]:
testmessage=x_test.iloc[0]

In [57]:
y_test

538571     1
759387     1
1061341    1
370775     1
388573     1
149096     1
1126807    1
99221      1
147786     1
643947     1
1241701    1
308431     1
327447     1
406250     1
951539     1
347495     1
517510     1
234837     1
19369      0
775729     1
1118001    1
1049345    1
41661      1
981502     1
2674       0
512290     1
1007011    1
534939     1
1035731    1
357589     1
          ..
1134255    1
253800     1
611853     1
929489     1
149785     1
1006556    1
683026     1
410593     1
588585     1
134332     1
540550     1
316993     1
388172     1
64469      1
854291     1
711969     1
56207      1
702982     1
552773     1
96851      1
689600     1
1196256    1
993109     1
277631     1
39245      0
32765      0
743001     1
349755     1
507148     1
869885     1
Name: new, Length: 268477, dtype: int64

In [58]:
predictions =mnb.predict(x_testcv)

In [59]:
predictions

array([1, 1, 1, ..., 1, 1, 1])

In [60]:
a=np.array(y_test)

In [61]:
a.shape - sum(a)

array([9521])

In [62]:
count=0
for i in range (len(predictions)):
    if predictions[i]==a[i]:
        count=count+1

count

265848

In [63]:
len(predictions)

268477

In [64]:
count*100/len(predictions)

99.02077272913509

In [65]:
tn, fp, fn, tp = confusion_matrix(a, predictions).ravel()

In [66]:
tp, fp

(258382, 2055)

In [67]:
tn, fn

(7466, 574)

In [68]:
(tp+tn)/(tn+ fp+ fn+ tp)

0.9902077272913509

In [70]:
tn/(tn+fn)

0.9286069651741293